Assignment is based on: http://neuralnetworksanddeeplearning.com/chap1.html <br\>
Code is taken from: https://github.com/MichalDanielDobrzanski/DeepLearningPython35

# Assignment 3 - Propose Your Own Assignment: Handwriting Detection

## Part A - Abstract

## Part B - Code Analysis

Here we will investigate each section of code that creates this neural network and interpret its meaning and utility.

In [49]:
class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

The above sets up the Network object and initializes its variables. Each network has a set of biases and weights which we initialize randomly to begin. <br\>
When creating a Network object, you initialize using an array (sizes). The first element in sizes is the input layer (and therefore has no associated bias), and represents the number of input nodes that this network has. Each subsequent element that precedes the final element represents the number of sigmoids in the corresponding hidden layers of the network. The final element in sizes is the number of output sigmoid neurons.<br\>
The biases are in the form of a y x 1 array (vector) of random values between 0 and 1. This initializes the biases to have a random start point for Stochastic Gradient Descent. Each layer of the network (besides the input layer) has an associated bias.<br\>
The weights are in the form of a y x x matrix. Each column of weights in the matrix represents the weights connecting the associated vector with the one following it. 

In [ ]:
def sigmoid(z):
    # this is the equation for the sigmoid function
    return 1.0/(1.0 + np.exp(-z))
def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

The above is the sigmoid function and its derivative. sigmoid takes the input (which for our purposes will be $z = w \cdot a + b$) and applies the sigmoid function, which can be written as: <br\>
$$ \sigma (z) \equiv \dfrac{1}{1 + e^{-z}} $$ With its derivative being the change in the sigmoid function.

In [ ]:
    def feedforward(self, a):
        # returns the output for input a
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a

The above is a function in the Network class. It calls the sigmoid function on the $z = w \cdot a + b$ step function to smooth out the graph and make the output vary from strictly 0 and 1.

In [ ]:
    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""

        training_data = list(training_data)
        n = len(training_data)

        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {} : {} / {}".format(j,self.evaluate(test_data),n_test));
            else:
                print("Epoch {} complete".format(j))

The above is also a part of the Network class, and is the Stochastic Gradient Descent algoritm. Gradient descent is an algorithm that minimizes the cost function of w, b (i.e. gets us closer to the correct solution). Stochastic gradient descent speeds this up by using smaller, randomized mini-batches to determine how to update weights and biases in the next step. <br\>
Each Epoch is a training step in the algorithm <br\>
eta is the learning rate, a small constant that mainitains a downward trajectory for the algorithm (continually making change in C smaller). At the same time, eta affects the size of the change in each training step, so one that is too small would be ineffective as there would be very little change. 

In [ ]:
    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

The above is also part of the Network class. It uses backpropogation to update the weights and biases in the network based on a mini-batch.

In [40]:
def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)
    
    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y)
                           for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
    
    def cost_derivative(self, output_activations, y):
        return (output_activations-y)
    
        

## Part C - Methods

We will now use this code and training data from the MNIST data set to create a neural network that can correctly identify hadwritten digits 0-9. For our data, the images of digits are 28x28 pixels in dimension and there are 10 possible types of digit (0-9). Therefore the input layer should always have 28 * 28 = 784 input neurons (one for each pixel) and 10 output neurons (one to represent each digit from 0 to 9).

### i. Two-Layer Network

In our first pass, we try a network with just an input layer and an output layer:

We then perform Stochastic Gradient Descent on the network using 10 epochs, a learning rate of .05, and a mini-batch size of 20:

After performing this 3 times, we take the best run which is:

In [51]:
# Go through and modify the algorithm input to output different values, 
# try to get closer to a good result

### ii. Three-Layer Network

### iii. Four-Layer Network

## Part D - Results

After trying many algorithms, our best result was: ****THE NETWORK AND ALGORITHM SPECIFICATIONS****, which outputted: ****THE OUTPUT**** after ****NUMBER OF EPOCHS**** epochs. 

In [52]:
##Graph of accuracy for each value

## Part E - Discussion

In [53]:
## DISCUSS RESULTS HERE

## Part F - References

Assignment is based on: http://neuralnetworksanddeeplearning.com/chap1.html <br\>
Code is taken from: https://github.com/MichalDanielDobrzanski/DeepLearningPython35